In [1]:
import pandas as pd

In [ ]:
sample_solution = pd.read_csv('../data/news/BBC News Sample Solution.csv')
train = pd.read_csv('../data/news/BBC News Train.csv')
test = pd.read_csv('../data/news/BBC News Test.csv')